<a href="https://colab.research.google.com/github/sara-gaballa/ADHD-detection-from-EEG-signals-using-ML-and-DL-models/blob/main/DATA2_with_new_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


##Features

In [ ]:
import numpy as np
from scipy import stats

# Define the feature extraction functions
def mean(data):
    return np.mean(data, axis=-1)

def std(data):
    return np.std(data, axis=-1)

def minim(data):
    return np.min(data, axis=-1)

def maxim(data):
    return np.max(data, axis=-1)

def median(data):
    return np.median(data, axis=-1)

def q1(data):
    return np.percentile(data, 25, axis=-1)

def q3(data):
    return np.percentile(data, 75, axis=-1)

def skewness(data):
    return stats.skew(data, axis=-1)

def kurtosis(data):
    return stats.kurtosis(data, axis=-1)

def extract_features(data):
    return np.concatenate((mean(data), std(data), minim(data), maxim(data),
                           median(data), q1(data), q3(data), kurtosis(data), skewness(data)), axis=-1)

##DATA 2 importing


In [ ]:
import os
import pandas as pd

folder_path = '/content/drive/MyDrive/Dataset 2/data files'

In [ ]:
pip install mat73

In [ ]:
import scipy.io
import os
import numpy as np
import mat73

# Prefix and suffix of the filenames
prefix = 'd'
suffix = '.mat'
# Range of file numbers to read
start_num = 1
end_num = 7

# Initialize an empty list to store the extracted data
data_list = []

# Loop over the desired file numbers and read the corresponding .mat files
for num in range(start_num, end_num+1):
  # Construct the filename
  filename = prefix + str(num) + suffix
  filepath = os.path.join(folder_path, filename)

  # Check if file exists
  if os.path.exists(filepath):
      # Load the .mat file and extract the desired variables
      mat = mat73.loadmat(filepath)

      # Append the extracted data to the list of data
      data_list.append(np.array(mat[prefix+str(num)]))
  else:
      print('File ' + filename + ' does not exist.')

File d1.mat does not exist.
File d2.mat does not exist.
File d3.mat does not exist.
File d4.mat does not exist.
File d5.mat does not exist.
File d6.mat does not exist.
File d7.mat does not exist.


In [ ]:
# Concatente all arrays
data = data_list[0]
for i in range(1, 7):
  data = np.concatenate((data, data_list[i]))
print(data.shape)
del(data_list)

IndexError: ignored

In [ ]:
control_trials = 10129
ADD_trials = 13031
ADHD_trials = 10742

HC = np.zeros(control_trials)
ADHD = np.ones(ADD_trials + ADHD_trials)

y = np.concatenate((HC, ADHD))
print(y.shape);

In [ ]:
from tqdm import tqdm_notebook
features2=[]
for trial in tqdm_notebook(data):
    features2.append(extract_features(trial))
features2=np.array(features2)
features2.shape


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features2, y, test_size=0.2, random_state=42)

##ML models

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, roc_auc_score
from sklearn.neural_network import MLPClassifier


In [ ]:
import time
start_time = time.time()
models = { 'Logistic Regression': LogisticRegression(),
           'Random Forest': RandomForestClassifier(),
           'KNeighbors':KNeighborsClassifier(),
            'Decision Tree':DecisionTreeClassifier(),
            'Gradient Boosting':GradientBoostingClassifier(),
            'SVC': SVC(),


}

params = { 'Logistic Regression': {'logisticregression__C': [0.1, 1, 10]},
           'Random Forest': {'randomforest__n_estimators': [10, 100, 1000],
                             'randomforest__max_depth': [None, 5, 10]},
           'KNeighbors':{'kneighbors__n_neighbors': [3, 5, 10], 'kneighbors__metric': ['euclidean', 'manhattan', 'minkowski']},
           'Decision Tree':{'decisiontree__max_depth': [None, 5, 10], 'decisiontree__criterion': ['gini', 'entropy']},
           'Gradient Boosting':{'gradientboosting__learning_rate': [0.5, 1, 1.5], 'gradientboosting__max_depth': [10, 15]},
            'SVC': {'svc__C': [0.1, 1, 10], 'svc__kernel': ['linear', 'rbf','poly']},


}

results = pd.DataFrame(columns=['Model', 'Parameters', 'Accuracy'])
for name, model in models.items():
  pipe = Pipeline([('scaler', StandardScaler()),(name.lower().replace(' ',''), model)])
  grid = GridSearchCV(pipe, params[name], cv=5, scoring='accuracy')

  # Fit the gridsearchcv on the train data
  grid.fit(x_train, y_train)

  # Predict on the test data
  y_pred = grid.predict(x_test)

  # Compute the accuracy score
  acc = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  auroc = roc_auc_score(y_test, y_pred)
  print(name, acc)
  # Append the results to the dataframe
  results = results.append({'Model': name, 'Parameters': grid.best_params_, 'Accuracy': acc,'Precision':precision,'Recall':recall,'F1':f1,'AURoc':auroc}, ignore_index=True)
test_4=pd.DataFrame({'actual':y_test, 'pred':y_pred})
results = results.sort_values(by='Accuracy', ascending=False)
end_time = time.time()

In [ ]:
results

In [ ]:
import time
start_time = time.time()
models = {
           'MLPClassifier':MLPClassifier()}


params = {
           'MLPClassifier':{'mlpclassifier__solver':['lbfgs','adam','sigmoid'],'mlpclassifier__alpha':[0.01,0.001,0.0001,0.00001,1e-5],'mlpclassifier__hidden_layer_sizes':[(10, 10),(10,5),(10,2),(5,5),(5,2)],'mlpclassifier__random_state':[1,None]}}


results = pd.DataFrame(columns=['Model', 'Parameters', 'Accuracy'])
for name, model in models.items():
  pipe = Pipeline([('scaler', StandardScaler()),(name.lower().replace(' ',''), model)])
  grid = GridSearchCV(pipe, params[name], cv=5, scoring='accuracy')
# Fit the gridsearchcv on the train data
  grid.fit(x_train, y_train)

  # Predict on the test data
  y_pred = grid.predict(x_test)

  # Compute the accuracy score
  acc = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  auroc = roc_auc_score(y_test, y_pred)
  print(name, acc)
  # Append the results to the dataframe
  results = results.append({'Model': name, 'Parameters': grid.best_params_, 'Accuracy': acc,'Precision':precision,'Recall':recall,'F1':f1,'AURoc':auroc}, ignore_index=True)
test_4=pd.DataFrame({'actual':y_test, 'pred':y_pred})
results = results.sort_values(by='Accuracy', ascending=False)
end_time = time.time()